In [1]:
import pandas as pd
import sqlite3

In [2]:
# Connecting to sql and cursor object
con = sqlite3.connect('./zippedData/im.db/im.db')
cursor_obj = con.cursor()


In [3]:
%%bash

sqlite3 './zippedData/im.db/im.db'
.schema


CREATE TABLE IF NOT EXISTS "movie_basics" (
"movie_id" TEXT,
  "primary_title" TEXT,
  "original_title" TEXT,
  "start_year" INTEGER,
  "runtime_minutes" REAL,
  "genres" TEXT
);
CREATE TABLE IF NOT EXISTS "directors" (
"movie_id" TEXT,
  "person_id" TEXT
);
CREATE TABLE IF NOT EXISTS "known_for" (
"person_id" TEXT,
  "movie_id" TEXT
);
CREATE TABLE IF NOT EXISTS "movie_akas" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "title" TEXT,
  "region" TEXT,
  "language" TEXT,
  "types" TEXT,
  "attributes" TEXT,
  "is_original_title" REAL
);
CREATE TABLE IF NOT EXISTS "movie_ratings" (
"movie_id" TEXT,
  "averagerating" REAL,
  "numvotes" INTEGER
);
CREATE TABLE IF NOT EXISTS "persons" (
"person_id" TEXT,
  "primary_name" TEXT,
  "birth_year" REAL,
  "death_year" REAL,
  "primary_profession" TEXT
);
CREATE TABLE IF NOT EXISTS "principals" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "person_id" TEXT,
  "category" TEXT,
  "job" TEXT,
  "characters" TEXT
);
CREATE TABLE IF NOT EXISTS "writers" (
"m

In [83]:
# gives all U.S movies
imdb_df = pd.read_sql ("""
SELECT a.movie_id, LOWER(primary_title) as title,
        start_year as year,
        CAST(runtime_minutes as INT) as runtime,
        genres,
        averagerating,
        numvotes
FROM movie_basics AS a 
INNER JOIN movie_akas AS b
ON a.movie_id = b.movie_id
INNER JOIN movie_ratings AS c
ON a.movie_id = c.movie_id
WHERE region = 'US' AND numvotes > 1000
ORDER by averagerating DESC, title 





""", con)

In [84]:
imdb_df.head()

,movie_id,title,year,runtime,genres,averagerating,numvotes
0,tt7131622,once upon a time ... in hollywood,2019,159.0,"Comedy,Drama",9.7,5600
1,tt7131622,once upon a time ... in hollywood,2019,159.0,"Comedy,Drama",9.7,5600
2,tt7738784,peranbu,2018,147.0,Drama,9.4,9629
3,tt2170667,wheels,2014,115.0,Drama,9.3,17308
4,tt2412748,saving mes aynak,2014,60.0,Documentary,9.1,1013


In [85]:
# Dropping Duplicates from data frame
imdb_df.drop_duplicates(inplace = True)

In [86]:
# Checking for Null Vallues
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5213 entries, 0 to 6852
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       5213 non-null   object 
 1   title          5213 non-null   object 
 2   year           5213 non-null   int64  
 3   runtime        5212 non-null   float64
 4   genres         5212 non-null   object 
 5   averagerating  5213 non-null   float64
 6   numvotes       5213 non-null   int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 325.8+ KB


In [87]:
# Dropping Null Values
imdb_df.dropna(inplace = True)

In [88]:
imdb_df

,movie_id,title,year,runtime,genres,averagerating,numvotes
0,tt7131622,once upon a time ... in hollywood,2019,159.0,"Comedy,Drama",9.7,5600
2,tt7738784,peranbu,2018,147.0,Drama,9.4,9629
3,tt2170667,wheels,2014,115.0,Drama,9.3,17308
4,tt2412748,saving mes aynak,2014,60.0,Documentary,9.1,1013
6,tt1795369,frankenstein,2011,130.0,Drama,9.0,1832
...,...,...,...,...,...,...,...
6845,tt2071491,jurassic shark,2012,75.0,"Action,Adventure,Comedy",1.6,3697
6846,tt1702443,justin bieber: never say never,2011,105.0,"Documentary,Music",1.6,74978
6849,tt4458206,code name: k.o.z.,2015,114.0,"Crime,Mystery",1.5,26723
6850,tt4009460,saving christmas,2014,79.0,"Comedy,Family",1.5,14221
